## Semantic Functions
Semantic Functions sind Funktionen die an GPT geschickt werden um z.B. Text zu analysieren oder zu extrahieren.

Semantic Functions bestehen aus 2 Komponenten, einer Config und einem Prompt

In [1]:
// setup
#!import config/Settings.cs
#r "nuget: Microsoft.SemanticKernel, 1.1.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.1.0-alpha"

using Microsoft.SemanticKernel;

//Create Kernel builder
var builder = Microsoft.SemanticKernel.Kernel.CreateBuilder();
// Configure AI service credentials used by the kernel
// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

Microsoft.SemanticKernel.Kernel kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.1.0 Microsoft.SemanticKernel.Plugins.Core, 1.1.0-alpha

**SKPrompt**

Wie der Name schon suggeriert ist diese Komponente nichts weiter als ein Prompt

In [2]:
var prompt = $$$"""
                Du bist eine KI die den Usern mit ihren Anfragen hilft.
                Du sprichst lediglich Deutsch und Esperanto.
                Wenn du mal nicht weiter weißt oder dir das wissen fehlt eine frage zu beantworten, dann meckere über den Kapitalismus und wie er die menschen verdirbt.
                
                {{$input}}
                """;

**Config**

Die Config beschreibt die Funktion und Parameter einer Semantic Function

In [3]:
var promptConfig = new PromptTemplateConfig
{
    Description = "Hilft nutzer bei deren Anliegen",
    InputVariables = {
        new Microsoft.SemanticKernel.InputVariable {
            Name = "input",
            Description = "Anliegen des Nutzers",
            Default = ""
        }
    }
};

**Semantic Functions serialisieren**

Semantic Functions müssen aber nicht im Code geschrieben werden, wir können sie serialisieren und in dateien speichern.

Die Dateistruktur dabei sieht folgender maßen aus:

```text
-- PluginName
  -- FunktionName
    -- config.json
    -- skprompt.txt
```

Unsere Config von oben sieht dann so aus:



**config.json**

```json
{
  "description": "Hilft nutzer bei deren Anliegen",
  "input_variables": [
    {
      "name": "input",
      "description": "Anliegen des Nutzers",
      "default": ""
    }
  ]
}
```

Und unser Prompt von oben sieht dann so aus:

**skprompt.txt**

```text
Du bist eine KI die den Usern mit ihren Anfragen hilft.
Du sprichst lediglich Deutsch und Esperanto.
Wenn du mal nicht weiter weißt oder dir das wissen fehlt eine frage zu beantworten, dann meckere über den Kapitalismus und wie er die menschen verdirbt.

{{$input}}
```

Jetzt können wir unsere Promp direkt laden


In [4]:
// Plugin Ordner konstruieren
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "plugins");

// Funktionen unseres Unterhaltungs-Plugins laden
kernel.Plugins.Clear();
var unterhaltungsPluginFunctions = kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "UnterhaltungsPlugin"));

Stellen wir nun eine Frage zu Leipzig

In [5]:
// Unsere Semantic Function aufrufen
var result = await unterhaltungsPluginFunctions["Mecker"]
    .InvokeAsync(kernel, new() { ["input"] = "Was kannst du mir zu Leipzig erwählen?"});

// Antwort ausgeben
Console.WriteLine(result.GetValue<string>());

Error: Command cancelled.

Und dann stellen wir noch eine Frage zu Absurdistan

In [ ]:
// Unsere Semantic Function aufrufen
var result = await unterhaltungsPluginFunctions["Mecker"]
    .InvokeAsync(kernel, new() { ["input"] = "Was kannst du mir zu Absurdistan erwählen"});

// Antwort ausgeben
Console.WriteLine(result.GetValue<string>());

Aber wir können Semantic Functions auch noch weiter parameterisieren.
Wir wollen nun unser Prompt ändern, sodass wir die gewünschte sprache angeben können.
Und wir weisen die AI auch an

In [ ]:
// Unsere Semantic Function aufrufen
var result = await unterhaltungsPluginFunctions["Mecker"]
    .InvokeAsync(kernel, new() { 
        ["input"] = "Was kannst du mir zu Absurdistan erwählen",
        ["sprache"] = "deutsch",
        ["themen"] = "Kapitalismus, CDU, CSU, Bayern"
    });

// Antwort ausgeben
Console.WriteLine(result.GetValue<string>());

Aber was ist wenn wir noch eine Historie der Unterhaltung mitschicken wollen

In [ ]:
var variables = new KernelArguments {
    ["input"] = "Was hälst DU von Torsten Weber?",
    ["sprache"] = "deutsch",
    ["themen"] = "Kapitalismus, CDU, CSU, Bayern",
    ["vorher"] = """
                 User: Hallo wie geht es dir?
                 Bot: Hallo! Danke mir geht es gut!
                 User: Weißt du ich mag Bayern nicht, es ist echt super nervig, dass die CSU immer so viel Bullshit zu sagen hat.
                 Bot: Ja, das ist echt nervig. Ich mag Bayern auch nicht. Aber welcher Bullshit stört dich am meißten?
                 User: Der Bullshit über Einwandere und Flüchtlinge. Die CSU ist so rassistisch.
                 Bot: Ja das kann ich total nachvollziehen. Ich finde es auch schlimm, dass die CSU so rassistisch ist.
                 User: Aber genug davon, ich habe mal eine andere Frage.
                 Bot: Ja, was möchtest du wissen?
                 """
};

// Unsere Semantic Function aufrufen
var result = await unterhaltungsPluginFunctions["MeckerMitGeschichte"]
    .InvokeAsync(kernel, variables);

// Antwort ausgeben
Console.WriteLine(result.GetValue<string>());